In [1]:
!pip install pydicom
!pip install opencv-python
import pandas as pd
import numpy as np
from pathlib import Path
import random
import pickle
import json
from pydicom import dcmread
from pydicom.data import get_testdata_file
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, GlobalAveragePooling2D, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
import tensorflow as tf
import time
import cv2
import gc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 99.5 MB/s eta 0:00:00


In [ ]:
home_folder=''

In [2]:
# Run iff executing in Google Colab
from google.colab import drive
drive.mount('/content/drive')
home_folder = '/content/drive/My Drive/Colab Notebooks/rsna-intercranial-aneurysm-detection/'

Mounted at /content/drive


In [ ]:
train = pd.read_csv('data/train.csv')
train_localizers = pd.read_csv('data/train_localizers.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4348 entries, 0 to 4347
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   SeriesInstanceUID                           4348 non-null   object
 1   PatientAge                                  4348 non-null   int64 
 2   PatientSex                                  4348 non-null   object
 3   Modality                                    4348 non-null   object
 4   Left Infraclinoid Internal Carotid Artery   4348 non-null   int64 
 5   Right Infraclinoid Internal Carotid Artery  4348 non-null   int64 
 6   Left Supraclinoid Internal Carotid Artery   4348 non-null   int64 
 7   Right Supraclinoid Internal Carotid Artery  4348 non-null   int64 
 8   Left Middle Cerebral Artery                 4348 non-null   int64 
 9   Right Middle Cerebral Artery                4348 non-null   int64 
 10  Anterior Communicating A

In [ ]:
train_localizers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2254 entries, 0 to 2253
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SeriesInstanceUID  2254 non-null   object
 1   SOPInstanceUID     2254 non-null   object
 2   coordinates        2254 non-null   object
 3   location           2254 non-null   object
dtypes: object(4)
memory usage: 70.6+ KB


In [ ]:
train_localizers.SeriesInstanceUID.value_counts()

SeriesInstanceUID
1.2.826.0.1.3680043.8.498.31629979420404800139928339434297456334    5
1.2.826.0.1.3680043.8.498.11527986509512933171256788651291467752    5
1.2.826.0.1.3680043.8.498.11292203154407642658894712229998766945    5
1.2.826.0.1.3680043.8.498.99028068919105186302294079606577228686    5
1.2.826.0.1.3680043.8.498.76928456732082261565048056589908832861    5
                                                                   ..
1.2.826.0.1.3680043.8.498.97970165518053195797247488050816887286    1
1.2.826.0.1.3680043.8.498.97975645720920888704056258456447231054    1
1.2.826.0.1.3680043.8.498.98066774276620948484052227331467077834    1
1.2.826.0.1.3680043.8.498.11079102674589284483149404820469555321    1
1.2.826.0.1.3680043.8.498.85592547875146602878105706110456654773    1
Name: count, Length: 1863, dtype: int64

In [ ]:
train_localizers.SOPInstanceUID.value_counts()

SOPInstanceUID
1.2.826.0.1.3680043.8.498.31624213251003577891438927087161881149    3
1.2.826.0.1.3680043.8.498.81825809496237506946898886010593439013    3
1.2.826.0.1.3680043.8.498.74508280112609225659548860717187732465    2
1.2.826.0.1.3680043.8.498.12451889826654706934051560968094172570    2
1.2.826.0.1.3680043.8.498.10330800208132441897498011905367111919    2
                                                                   ..
1.2.826.0.1.3680043.8.498.44185937258859556516668591199094598570    1
1.2.826.0.1.3680043.8.498.35987680962070754560160756829496799559    1
1.2.826.0.1.3680043.8.498.11937074874094187336286047915868824604    1
1.2.826.0.1.3680043.8.498.10433249525306561584387515837440191252    1
1.2.826.0.1.3680043.8.498.53868409774237283281776807176852774246    1
Name: count, Length: 2214, dtype: int64

In [ ]:
train_localizers.SOPInstanceUID.nunique()

2214

In [ ]:
train_localizers.head()

,SeriesInstanceUID,SOPInstanceUID,coordinates,location
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,"{'x': 258.3621186176837, 'y': 261.359900373599}",Other Posterior Circulation
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,"{'x': 194.87253141831238, 'y': 178.32675044883...",Right Middle Cerebral Artery
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,"{'x': 189.23979878597123, 'y': 209.19184886465...",Right Middle Cerebral Artery
3,1.2.826.0.1.3680043.8.498.10030095840917973694...,1.2.826.0.1.3680043.8.498.75217084841854214544...,"{'x': 208.2805049088359, 'y': 229.78962131837307}",Right Infraclinoid Internal Carotid Artery
4,1.2.826.0.1.3680043.8.498.10034081836061566510...,1.2.826.0.1.3680043.8.498.71237104731452368587...,"{'x': 249.86745590416498, 'y': 220.623044646393}",Anterior Communicating Artery


In [ ]:
train_localizers['coordinates_dict'] = train_localizers['coordinates'].str.replace("'", '"').apply(json.loads)
train_localizers['frame'] = train_localizers['coordinates_dict'].apply(lambda d: d['f'] if 'f' in d else -1)
train_df = train_localizers[['SeriesInstanceUID', 'SOPInstanceUID', 'frame']].copy()
train_df['aneurysm_present'] = 1

In [ ]:
train_df.head()

,SeriesInstanceUID,SOPInstanceUID,frame,aneurysm_present
0,1.2.826.0.1.3680043.8.498.10005158603912009425...,1.2.826.0.1.3680043.8.498.10775329348174902199...,-1,1
1,1.2.826.0.1.3680043.8.498.10022796280698534221...,1.2.826.0.1.3680043.8.498.53868409774237283281...,-1,1
2,1.2.826.0.1.3680043.8.498.10023411164590664678...,1.2.826.0.1.3680043.8.498.24186535344744886473...,-1,1
3,1.2.826.0.1.3680043.8.498.10030095840917973694...,1.2.826.0.1.3680043.8.498.75217084841854214544...,-1,1
4,1.2.826.0.1.3680043.8.498.10034081836061566510...,1.2.826.0.1.3680043.8.498.71237104731452368587...,-1,1


#### Select training images

There are a huge amount of training images - over 1 million individual images with a total size of over 300 GB. This very large size presents a major challenge given the computational resources, storage, and RAM I have available. I have decided to use only a small subset of those images for training.

Of note is that out of those 1 million images, only 2,214 actually contain an aneurysm. So I will include all of those images, but only a small subset of the other ~1 million images.

There are 4,348 individual patients in the dataset, many with hundreds of images in their imaging sets. However, as we can see from `train_localizers.SeriesInstanceUID.value_counts()`, no patient has more than 5 images containing an aneurysm.

So, to include all of the images with an aneurysm, but also get a wide and diverse dataset, while also limiting the size of the overall training set to make it more manageable, I have implemented the following algorithm:

- Iterate through all 4,348 patients.
- For each patient, select 5 images from their image folder (if the patient has less than 5 images total, select all images)
- For these 5 images, first, select all images that contain an aneurysm by cross-referencing with `train_localizers`.
- Then, for the remaining images, randomly select from that patient's remaining images.
- Add the file path for these images to `train_image_paths`, and the label for the corresponding image to `train_image_labels` (`1` for aneurysm present and `0` for aneurysm absent
- This will select a total of `4,348 * 5 = 21,740` images to use for training. (The actual number will be slightly less due to not all patients having 5 or more images).

This algorithm is implemented below:

In [ ]:
random.seed(42)
start_time = time.time()
total_images_skipped = 0
for series_id in train['SeriesInstanceUID']:
    # Select a random sample of five images from each series
    # Add to train_df with frame = -2 to indicate aneurysm absent
    # Check to ensure the series + image id is not already in train_df first
    series_directory = Path(home_folder + 'series/' + series_id)
    all_images_in_series = [file.stem for file in series_directory.iterdir()]
    frame = -1
    for i in range(0, min(5, len(all_images_in_series))):
        randomly_chosen_image_id = random.choice(all_images_in_series)
        if randomly_chosen_image_id not in train_df.SOPInstanceUID:
            read_image = dcmread(home_folder + 'series/' + series_id + '/' + randomly_chosen_image_id + '.dcm', stop_before_pixels=True)
            if hasattr(read_image, "NumberOfFrames"):
                # (A few images seem to have wonky metadata, just skip those)
                if not isinstance(read_image.NumberOfFrames, int):
                    total_images_skipped += 1
                    continue
                num_frames = int(read_image.NumberOfFrames)
                if num_frames > 1:
                    frame = random.randint(0, num_frames - 1)
            train_df.loc[len(train_df)] = [series_id, randomly_chosen_image_id, frame, 0]

print(f'Image selection algorithm complete. Elapsed time: {(time.time() - start_time)} seconds. Total images skipped: {total_images_skipped}')

Image selection algorithm complete. Elapsed time: 72.47417187690735 seconds. Total images skipped: 5


In [ ]:
train_df.sample(20)

,SeriesInstanceUID,SOPInstanceUID,frame,aneurysm_present
17073,1.2.826.0.1.3680043.8.498.67739094738603286189...,1.2.826.0.1.3680043.8.498.81836229612336180441...,-1,0
17952,1.2.826.0.1.3680043.8.498.73449945381434949471...,1.2.826.0.1.3680043.8.498.10289516985798128548...,-1,0
3539,1.2.826.0.1.3680043.8.498.10752089895877999881...,1.2.826.0.1.3680043.8.498.60084404100479463690...,-1,0
10748,1.2.826.0.1.3680043.8.498.29530939414757604033...,1.2.826.0.1.3680043.8.498.34833252557548479658...,-1,0
1515,1.2.826.0.1.3680043.8.498.59538010309137561846...,1.2.826.0.1.3680043.8.498.61201710808940352746...,69,1
8679,1.2.826.0.1.3680043.8.498.17415277997649872560...,1.2.826.0.1.3680043.8.498.12162973189564091854...,-1,0
4771,1.2.826.0.1.3680043.8.498.11507545288604071234...,1.2.826.0.1.3680043.8.498.77016982829356506398...,-1,0
18553,1.2.826.0.1.3680043.8.498.76928456732082261565...,1.2.826.0.1.3680043.8.498.13176661637246944064...,-1,0
14346,1.2.826.0.1.3680043.8.498.50462346896825686194...,1.2.826.0.1.3680043.8.498.56703616746011447418...,-1,0
12036,1.2.826.0.1.3680043.8.498.36812477383471426955...,1.2.826.0.1.3680043.8.498.23609312081130299094...,-1,0


In [ ]:
train_df.to_csv('data_gen/train_df.csv', index=False)

In [ ]:
train_df_loaded = pd.read_csv('data_gen/train_df.csv')

Next, we want to split the data into a train set and a validation set. We use sklearn's `train_test_split`, with a test_size of 0.15 to put 15% of the data into the validation set and leave the other 85% in the training set. Very importantly, we use `stratify = training_image_labels` to ensure equivalent proportions of positive and negative images in both the train and validation sets. Positive images (i.e. images with aneurysms) still make up only ~10% of the selected training images, and with such imbalanced classes it is crucial that the validation set matches the overall proportions.

In [ ]:
train_df_X = list(zip(train_df_loaded.SeriesInstanceUID, train_df.SOPInstanceUID, train_df.frame))
train_df_y = train_df['aneurysm_present']

X_train_paths, X_val_paths, y_train, y_val = train_test_split(
    train_df_X, train_df_y, test_size = 0.15, random_state=42, stratify = train_df_y
)

In [ ]:
len(X_train_paths), len(X_val_paths), len(y_train), len(y_val)

(19295, 3406, 19295, 3406)

#### Preprocessing image data

Here comes the trickiest part. We have a wide variety of different kinds of medical images, with a wide variety of different dimensions. Many but not all of the images are square. To feed this into a convolutional network, it would be best to ensure a standardized format (that is, equivalent x and y dimensions for all images).

The images are in the DICOM medical imaging format, so we use `pydicom` to read the image data.

We then need to convert all images into standard dimensions, e.g. 512x512, by some combination of scaling/padding/cropping.

We also should rescale the pixel values to be between zero and one. (The images are all greyscale)

Finally, we need to convert each processed image into a 2d numpy array or tensor of `float16`s. `float16` is the smallest floating point format which will greatly reduce the total size of the images. All of these 2d arrays/tensors will then need to be combined into a single large array to then feed as input to our convolutional neural network in the next phase.

(Additionally, we strongly consider data augmentation, where we create rotated/distorted/blurred versions of our training data. This would likely be very helpful in this case because there are so few example images that contain aneurysms (just over 2,000). However, due to my limited computational resources, I will skip this step for the time being. Potentially, I will revisit this later when trying to improve performance of the network.)

The preprocessing phase is implemented below:

In [10]:
processed_image_dim = 512
processed_image_dimensions = (processed_image_dim, processed_image_dim)

def preprocess_image(image_info_tuple):
    series_id, image_id, frame = image_info_tuple
    image_path = home_folder + 'series/' + series_id + '/' + image_id + '.dcm'

    # Steps:
    # Read DICOM image and convert to numpy pixel array
    pixel_array = dcmread(image_path).pixel_array.astype(np.float32)
    if frame == -1:
        assert len(pixel_array.shape) == 2, f'frame == -1 but pixel array shape is {pixel_array.shape}'
    else:
        assert len(pixel_array.shape) == 3, f'frame == {frame} but pixel array shape is {pixel_array.shape}'
        pixel_array = pixel_array[frame]

    # - Rescale pixel values to range [0, 1]
    pixel_min, pixel_max = pixel_array.min(), pixel_array.max()
    if pixel_max == pixel_min:
        pixel_array_scaled = np.zeros_like(pixel_array, dtype=np.float32)
    else:
        pixel_array_scaled = (pixel_array - pixel_min) / (pixel_max - pixel_min)

    # - Pad/scale to 512x512
    try:
        x_dim, y_dim = pixel_array_scaled.shape
    except Exception as e:
        print(f'Exception due to unexpected shape {pixel_array_scaled.shape}. Image path: {image_path}')

    if x_dim != y_dim:
        max_dim = max(x_dim, y_dim)

        pad_x = max_dim - x_dim
        pad_left = pad_x // 2
        pad_right = pad_x - pad_left

        pad_y = max_dim - y_dim
        pad_top = pad_y // 2
        pad_bottom = pad_y - pad_top

        previous_shape = pixel_array_scaled.shape
        pixel_array_scaled = np.pad(
            pixel_array_scaled,
            pad_width=((pad_left, pad_right), (pad_top, pad_bottom)),
            mode='constant',
            constant_values=0
        )
        x_dim, y_dim = pixel_array_scaled.shape
        assert x_dim == y_dim, f'x_dim should equal y_dim. x_dim == {x_dim}, y_dim == {y_dim}. Previous shape: {previous_shape}'

    if x_dim != processed_image_dim:
        if x_dim > processed_image_dim:
            pixel_array_scaled = cv2.resize(pixel_array_scaled, processed_image_dimensions, cv2.INTER_AREA)
        if x_dim < processed_image_dim:
            pixel_array_scaled = cv2.resize(pixel_array_scaled, processed_image_dimensions, cv2.INTER_CUBIC)

    # Convert to float16 to reduce memory usage
    return pixel_array_scaled.astype(np.float16)

In [ ]:
start_time = time.time()
X_val = np.asarray([preprocess_image(image_info_tuple) for image_info_tuple in X_val_paths])
print(f'Completed preprocessing {len(X_val)} validation images. Elapsed time: {time.time() - start_time} seconds.')

Completed preprocessing 3406 validation images. Elapsed time: 58.8567008972168 seconds.


In [3]:
X_val_file_path = home_folder + 'data_gen/X_val.pkl'
X_train_file_path = home_folder + 'data_gen/X_train.pkl'

In [ ]:
with open(X_val_file_path, 'wb') as file:
    pickle.dump(X_val, file)

In [ ]:
start_time = time.time()
X_train = np.asarray([preprocess_image(image_info_tuple) for image_info_tuple in X_train_paths])
print(f'Completed preprocessing {len(X_train)} training images. Elapsed time: {time.time() - start_time} seconds.')

C:\ProgramData\anaconda3\envs\tensorflow\Lib\site-packages\pydicom\pixels\utils.py:222: UserWarning: A value of 'None' for (0028,0008) 'Number of Frames' is invalid, assuming 1 frame
  warn_and_log(


Completed preprocessing 19295 training images. Elapsed time: 380.75668811798096 seconds.


In [ ]:
with open(X_train_file_path, 'wb') as file:
    pickle.dump(X_train, file)

In [ ]:
y_val = y_val.tolist()
y_train = y_train.tolist()

In [4]:
y_val_file_path = home_folder + 'data_gen/y_val.txt'
y_train_file_path = home_folder + 'data_gen/y_train.txt'

In [ ]:
with open(y_val_file_path, 'w', encoding='utf-8') as file:
    json.dump(y_val, file, indent=4, ensure_ascii=False)

with open(y_train_file_path, 'w', encoding='utf-8') as file:
    json.dump(y_train, file, indent=4, ensure_ascii=False)

#### Loading saved images

The preprocessing steps were completed above, and then the training and validation data were saved as `.pkl` and `.txt` files. Below, we load the saved data into `x_train_loaded`, `x_val_loaded`, `y_train_loaded`, and `y_val_loaded`.

In [6]:
with open(y_val_file_path, 'r', encoding='utf-8') as file:
    y_val_loaded = json.loads(file.read())

with open(y_train_file_path, 'r', encoding='utf-8') as file:
    y_train_loaded = json.loads(file.read())

y_val_loaded = np.array(y_val_loaded)
y_train_loaded = np.array(y_train_loaded)

In [7]:
with open(X_val_file_path, 'rb') as file:
    X_val_loaded = pickle.load(file)

In [8]:
with open(X_train_file_path, 'rb') as file:
    X_train_loaded = pickle.load(file)

#### Model Architecture

Below, I define the architecture of my convolutional neural network. For this architecture, I started with the sample architecture given by Professor Josh Hug in Lecture 22.2:

In [ ]:
# Do not execute: for demonstration purposes only
model = Sequential([
    Input(shape=(28, 28, 1)),
    Conv2D(filters=32, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=64, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),
    Conv2D(filters=128, kernel_size=3, activation="relu"),
    Flatten(),
    Dense(10, activation="softmax")
])

Many changes needed to be made to this architecture for this problem, however:

- With input images of `512x512` instead of `28x28`, the input shape needs to be changed to `(512, 512, 1)`
- As this is a binary classification tasks rather than a classification task with 10 classes, the final layer needs to be changed to a dense layer with 1 node and a sigmoid activation function
- Because the images are so much larger, I added in a couple more convolutional layers to reduce the size of the feature space.
- I also added in a couple Dropout layers to hopefully help with overfitting
- Finally, after noticing that the final `Flatten()` layer was generating a huge number of parameters, I switched it out for a `GlobalAveragePooling2D()` layer based on some recommendations I found online.
- I also added `BatchNormalization` layers to some of the later convolutional layers.

My chosen architecture is shown below. Note that this is only my initial architecture and I will try to further tune it later. But for now this seemed like a reasonable starting point.

In [11]:
model = Sequential([
    Input(shape=(processed_image_dim, processed_image_dim, 1)),

    Conv2D(filters=32, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=64, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=128, kernel_size=3, use_bias=False),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D(pool_size=2),
    Dropout(0.25),

    Conv2D(filters=256, kernel_size=3, activation="relu"),
    MaxPooling2D(pool_size=2),

    Conv2D(filters=256, kernel_size=3, use_bias=False),
    BatchNormalization(),
    Activation('relu'),

    GlobalAveragePooling2D(),
    Dropout(0.5),

    Dense(1, activation="sigmoid"),
])
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 510, 510, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 124, 124, 128)  │        73,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 124, 124, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 124, 124, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 60, 60, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 30, 30, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 256)    │       589,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 979,329 (3.74 MB)

 Trainable params: 978,561 (3.73 MB)

 Non-trainable params: 768 (3.00 KB)

In [12]:
training_history_file_path = home_folder + 'data_gen/training_history.txt'
model_save_folder = home_folder + 'data_gen/saved_model'

In [ ]:
history = model.fit(
    x = X_train_loaded,
    y = y_train_loaded,
    epochs = 5,
    validation_data = (X_val_loaded, y_val_loaded)
)

# Save learned weights and training history
model.save(model_save_folder)
with open(training_history_file_path, 'w', encoding='utf-8') as file:
    json.dump(history.history, file, indent=4, ensure_ascii=False)